In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/29155?type=Model&format=SafeTensor' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = 'aniflatmixAnimeFlatColorStyle_v10.safetensors' #@param {type:"string"}

#@markdown -----
#@markdown *추가익스텐션*
#@markdown - ControlNet/ADetailer  

Workspace = 'ui'
NotebookVersion = 'simple'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion/google'

!apt -y install -qq aria2
!pip install xformers==0.0.20 triton==2.0.0 -U

%cd /content
!git clone https://github.com/ninjaneural/colabui /content/{Workspace}

# 구글드라이브 연결
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/link_google_drive.sh -O /content/link_google_drive.sh
!bash /content/link_google_drive.sh {Workspace} {Google_Drive} {Google_Drive_Dir}

%cd /content/{Workspace}
!git clone -b stable1 https://github.com/ninjaneural/controlnet ./extensions/controlnet
!git clone https://github.com/Bing-su/adetailer ./extensions/adetailer./extensions/adetailer
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/config.json

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/ESRGAN -o 4x-UltraSharp.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/{Workspace}/models/VAE -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/63006?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o LowRA.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16" -d /content/{Workspace}/models/Lora -o blindbox_V1Mix.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d /content/{Workspace}/embeddings -o bad-hands-5.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o bad_prompt_version2.pt

import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()

  #ControlNet
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d ./extensions/controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d ./extensions/controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d ./extensions/controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d ./extensions/controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
  !aria2c --console-log-level=error --download-result=hide -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d ./extensions/controlnet/models -o diff_control_sd15_temporalnet_fp16.safetensors

threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

!python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark